In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import h5py
import numpy as np
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset


In [5]:
#work with body vocab

#load data from hdf5 file and create a dataset
with h5py.File("data/nq17-23_1min_candle_seq_463.hdf5", "r") as f:
    total_records = f["data"].shape[0]
    # start_index = int(total_records*0.55)
    # stop_index = int(total_records*0.65)
    dataset = f["data"][:]
    dataset = torch.from_numpy(dataset).long()
#load index to candlestick mapping from hdf5 file

    index_to_candle = f["index_to_candle"][:]
    

candle_seq_len = 16

dataset = dataset.unfold(0, candle_seq_len, 1)
input_dataset = dataset[:, :-1]
target_dataset = dataset[: , 1:]

dataset.shape


torch.Size([2372821, 16])

In [6]:
input_dataset[0], target_dataset[0]

(tensor([ 0,  1,  2,  3,  4,  5,  6,  2,  7,  7,  8,  8,  9, 10,  2]),
 tensor([ 1,  2,  3,  4,  5,  6,  2,  7,  7,  8,  8,  9, 10,  2,  8]))

In [22]:
torch.manual_seed(42)
split_idx = int(len(dataset)*0.8)
train_data = input_dataset[:split_idx]
train_targets = target_dataset[:split_idx]
test_data = input_dataset[split_idx:]
test_targets = target_dataset[split_idx:]


train_dataset = TensorDataset(train_data, train_targets)
test_dataset = TensorDataset(test_data, test_targets)

batch_size = 256
#load data into dataloader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,num_workers=4)


In [8]:


# function to restore candles from their codes and plot candlestick chart to writer
import plotly.graph_objects as go
import io
from PIL import Image

def write_charts_to_TB(name,writer, targets, outputs, epoch, index_map):
    
    tick_size = 0.25
    #convert outputs logit to candlestick codes
    outputs = outputs.softmax(dim=1).argmax(dim=1)

    def full_candle_restore(candle_index:(np.array,torch.Tensor), start_price:float = 1000, number_of_candles = None, index_map = index_to_candle) -> pd.DataFrame:
        '''
        Restore full candle from index of candle and start price
        '''
        if isinstance(candle_index, torch.Tensor):
            candle_index = candle_index.numpy()
        if number_of_candles is not None:
            candle_index = candle_index[:number_of_candles]
        
        candles = []
        for idx, cdl_idx in enumerate(candle_index):
            candle = {}
            if idx == 0:
                candle['open'] = start_price
            else:
                candle['open'] = candles[-1]['close']
            top_wick, body, bottom_wick = index_map[cdl_idx]
            close = candle['open'] + body * tick_size
            high = close + top_wick * tick_size if close > candle['open'] else candle['open'] + top_wick * tick_size
            low = candle['open'] - bottom_wick * tick_size if close > candle['open'] else close - bottom_wick * tick_size
            candle['high'] = high
            candle['low'] = low
            candle['close'] = close
            candles.append(candle)
    
        return pd.DataFrame(candles)

    original_candles = full_candle_restore(targets)
    predicted_candles = full_candle_restore(outputs)
    fig1 = go.Figure(data=[go.Candlestick(x=original_candles.index,
                open=original_candles['open'],
                high=original_candles['high'],
                low=original_candles['low'],
                close=original_candles['close'])])
    #increase chart size
    fig1.update_layout(height=600, width=1200)
    fig1.update_layout(xaxis_rangeslider_visible=False)
    

    fig2 = go.Figure(data=[go.Candlestick(x=predicted_candles.index,
                    open=predicted_candles['open'],
                    high=predicted_candles['high'],
                    low=predicted_candles['low'],
                    close=predicted_candles['close'])])
    fig2.update_layout(height=600, width=1200)
    fig2.update_layout(xaxis_rangeslider_visible=False)
    
    #convert figures to image and write to tensorboard
    fig1_bytes = fig1.to_image(format="png")
    fig2_bytes = fig2.to_image(format="png")

    # Преобразуем байтовые данные в массив NumPy
    fig1_image = np.array(Image.open(io.BytesIO(fig1_bytes)))
    fig2_image = np.array(Image.open(io.BytesIO(fig2_bytes)))

    # Преобразуем массивы NumPy в тензоры PyTorch
  

    # Добавляем изображения в TensorBoard
    writer.add_image(f'{name}_original', fig1_image, epoch, dataformats='HWC')
    writer.add_image(f'{name}_predicted', fig2_image, epoch, dataformats='HWC')

def write_charts_to_sceeen(name,writer, targets, outputs, epoch, index_map):
    
    tick_size = 0.25
    #convert outputs logit to candlestick codes
    outputs = outputs.softmax(dim=1).argmax(dim=1)

    def full_candle_restore(candle_index:(np.array,torch.Tensor), start_price:float = 1000, number_of_candles = None, index_map = index_to_candle) -> pd.DataFrame:
        '''
        Restore full candle from index of candle and start price
        '''
        if isinstance(candle_index, torch.Tensor):
            candle_index = candle_index.numpy()
        if number_of_candles is not None:
            candle_index = candle_index[:number_of_candles]
        
        candles = []
        for idx, cdl_idx in enumerate(candle_index):
            candle = {}
            if idx == 0:
                candle['open'] = start_price
            else:
                candle['open'] = candles[-1]['close']
            top_wick, body, bottom_wick = index_map[cdl_idx]
            close = candle['open'] + body * tick_size
            high = close + top_wick * tick_size if close > candle['open'] else candle['open'] + top_wick * tick_size
            low = candle['open'] - bottom_wick * tick_size if close > candle['open'] else close - bottom_wick * tick_size
            candle['high'] = high
            candle['low'] = low
            candle['close'] = close
            candles.append(candle)
    
        return pd.DataFrame(candles)

    original_candles = full_candle_restore(targets)
    predicted_candles = full_candle_restore(outputs)
    fig1 = go.Figure(data=[go.Candlestick(x=original_candles.index,
                open=original_candles['open'],
                high=original_candles['high'],
                low=original_candles['low'],
                close=original_candles['close'])])
    #increase chart size
    fig1.update_layout(height=600, width=1200)
    fig1.update_layout(xaxis_rangeslider_visible=False)
    

    fig2 = go.Figure(data=[go.Candlestick(x=predicted_candles.index,
                    open=predicted_candles['open'],
                    high=predicted_candles['high'],
                    low=predicted_candles['low'],
                    close=predicted_candles['close'])])
    fig2.update_layout(height=600, width=1200)
    fig2.update_layout(xaxis_rangeslider_visible=False)
    
    fig1.show()
    fig2.show()



In [9]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[batch_size, seq_len, embedding_dim]``
        """
        x = x + self.pe[:x.size(1)].transpose(0, 1)  # Изменение размеров для соответствия формату batch_first
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoder = PositionalEncoding(embed_dim, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.output_layer = nn.Linear(embed_dim, vocab_size)
        self.embed_dim = embed_dim
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0, std=0.02)  

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.embed_dim)
        # print('Enbedding shape:', src.shape)
        src = self.pos_encoder(src)
        # print('Positional encoding shape:', src.shape)
        output = self.transformer_encoder(src)
        # print('Transformer encoder output shape:', output.shape)
    
        output = self.output_layer(output)
        # print('Output layer shape:', output.shape)
        return output #return only last output


In [10]:
#set of model parameters
vocab_size = dataset.max() + 1
embed_dim = 64
num_heads = 8
num_layers = 1
dropout = 0.1

#check if we have CUDA or MPS and setup respecive device, if not CUDA nor MPS is available, then use CPU
def init_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')

    #device = torch.device('cpu')

    print(f"Device to be used: {device}")
    #Initialize model
    torch.manual_seed(42)
    model = TransformerModel(vocab_size, embed_dim, num_heads, num_layers, dropout)

    model = model.to(device)
    #print(model)
    #print model device
    next(model.parameters()).device
    return model, device
    

In [8]:
# # test model forward pass
# for idx, (data, target) in enumerate(train_loader):
    # model, device = init_model()
    # data = data.to(device)
    # target = target.to(device)
    # output = model(data)
    # print(f'Output shape: {output.shape}')
    # print(f'Input data shape: {data.shape}')
    # print(f'Target shape: {target.shape}')
    # break

In [11]:
#define writer for tensorboard
import os
delete_logs = True
if delete_logs:
    #use python os package to delete logs including files in subfolders and subfolders itself
    for root, dirs, files in os.walk('./runs/nq_llm_body_0'):
        for file in files:
            os.remove(os.path.join(root, file))
        for dir in dirs:
            for fils in os.listdir(os.path.join(root, dir)):
                os.remove(os.path.join(root, dir, fils))
            os.rmdir(os.path.join(root, dir))   
    
writer = SummaryWriter('runs/nq_llm_body_0')

In [12]:
#initialize model, loss function and optimizer
model, device = init_model()
loss_fn = nn.CrossEntropyLoss(reduction='none')
optimizer = optim.AdamW(model.parameters(), lr=0.001 )  #, weight_decay=1e-5)
seq_len = candle_seq_len - 1

loss_weights = torch.ones(seq_len).to(device)
loss_weights[-1] = 60.0

best_vloss = float('inf')

Device to be used: mps


In [11]:
# #caclulate number of parameters in the model
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)
# count_parameters(model)

In [38]:
#restore model  and optimizer state from checkpoint './models/nq-llm_0.pth'

checkpoint = torch.load('./models/nq-llm_0_1.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
best_vloss = checkpoint['vloss']
#print optimizer state
for param_group in optimizer.param_groups:
    print(param_group['lr'])
    print(param_group['weight_decay'])
print(epoch)
print(best_vloss)






0.0001
0.0001
62
3.1444900892956227


In [39]:
# direct adjust optimizer learning rate and weight decay

for g in optimizer.param_groups:
    g['lr'] = 1e-4
    g['weight_decay'] = 1e-5

loss_weights[-1] = 10.0

In [40]:
start_epoch = 108
num_epochs = 200


# Main training loop
try:
    for epoch in range(start_epoch,num_epochs):
        # Train the model
        model.train()
        epoch_loss = 0
        for batch_idx, (data, labels) in enumerate(train_loader):
            data = data.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(data)

            #loss caclulation with weighted last candle

            loss = loss_fn(outputs.view(-1, vocab_size), labels.view(-1))
            loss = loss.view(-1, seq_len)
            weighted_loss = loss * loss_weights
            loss = weighted_loss.mean()

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():,.6f}', end='\r', flush=True)
      
        
            

        #add weights and biases to tensorboard
        weights = {}
        biases = {}
        grads = {}
        for name, param in model.named_parameters():
            if 'weight' in name:
                writer.add_histogram(f'weights/{name}', param, epoch)
            elif 'bias' in name:
                writer.add_histogram(f'biases/{name}', param, epoch)
            if param.grad is not None:
                writer.add_histogram(f'grads/{name}', param.grad, epoch)



        # Test the model
        
        model.eval()
        vepoch_loss = 0
        with torch.no_grad():
            correct = 0
            total = 0
            for vbatch_idx, (vdata, vlabels) in enumerate(test_loader):
                vdata = vdata.to(device)
                vlabels = vlabels.to(device)
                voutputs = model(vdata)
                #loss caclulation with weighted last candle
                vloss = loss_fn(voutputs.view(-1, vocab_size), vlabels.view(-1))
                vloss = vloss.view(-1, seq_len)
                vweighted_loss = vloss * loss_weights
                vloss = vweighted_loss.mean()
                vepoch_loss += vloss.item()

                #calculate accuracy
                last_redicted_candle = voutputs[:,-1,:].softmax(dim=1).argmax(dim=1)
                last_actual_candle = vlabels[:,-1]
                correct += (last_redicted_candle == last_actual_candle).sum().item()
                total += last_actual_candle.size(0)
                accuracy = correct / total * 100
                
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{vbatch_idx + 1}/{len(test_loader)}], Validation Loss: {vloss.item():,.6f}, Validation accuracy: {accuracy:.2f}%', end='\r', flush=True)
    

        # Save the model checkpoint if validation loss is less than best validation loss
        if vepoch_loss/len(test_loader) < best_vloss:
            best_vloss = vepoch_loss/len(test_loader)
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'vocab_size': vocab_size,
                'embed_dim': embed_dim,
                'num_heads': num_heads,
                'num_layers': num_layers,
                'dropout': dropout,
                'vloss': best_vloss,
            
                }, './models/nq-llm_0_1.pth')
            lr = next(iter(optimizer.param_groups))['lr']
            weight_decay = next(iter(optimizer.param_groups))['weight_decay']
            print(f"Model saved at epoch {epoch+1} with validation loss {vepoch_loss/len(test_loader):.6f} Learning rate: {lr:.2e} Weight decay: {weight_decay:.2e} ")
        #else - restore the model from previous checkpoint and reduce learning rate 5 times and increase weight decay 50%
        # else:
        #     checkpoint = torch.load('./models/nq-lstm.pth')
        #     model.load_state_dict(checkpoint['model_state_dict'])
        #     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            
        #     for g in optimizer.param_groups:
        #         g['lr'] = g['lr'] * 0.3
        #         g['weight_decay'] = g['weight_decay'] * 1.1
        #     print(f"Model restored from epoch {epoch} with validation loss {best_vloss/len(test_loader)}")
        #     print(f"Learning rate reduced to {g['lr']} and weight decay increased to {g['weight_decay']}")

        

        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {epoch_loss / len(train_loader):,.6f}, Validation Loss: {vepoch_loss / len(test_loader):,.6f}, Validation accuracy: {accuracy:.2f}% ')
        writer.add_scalars('Loss', {'Train': epoch_loss / len(train_loader), 'Test': vepoch_loss / len(test_loader)}, epoch)
        writer.add_scalar('Accuracy', accuracy, epoch)
        
        
        # get actual lr and weight decay from optimizer and write them to tensorboard
        lr = next(iter(optimizer.param_groups))['lr']
        weight_decay = next(iter(optimizer.param_groups))['weight_decay']
        writer.add_scalar('Learning rate', lr, epoch)
        writer.add_scalar('Weight decay', weight_decay, epoch)
        
        #write charts to tensorboard
        write_charts_to_TB('Test data sample',writer, vlabels[0].cpu(), voutputs[0].cpu(), epoch, index_to_candle)
        write_charts_to_TB('Test predicted candles sequence',writer, vlabels[:60,-1].cpu(), voutputs[:60,-1,:].cpu(), epoch, index_to_candle)
        

except KeyboardInterrupt:
    print(f"Interrupted at epoch {epoch} with Training loss {epoch_loss:,.6f} and Validation loss {vepoch_loss:,.6f}")
finally:
    writer.close()
    



Epoch [109/200], Train Loss: 2.856882, Validation Loss: 3.144511, Validation accuracy: 2.94% 
Epoch [110/200], Train Loss: 2.856535, Validation Loss: 3.144938, Validation accuracy: 2.97% 
Epoch [111/200], Train Loss: 2.856684, Validation Loss: 3.145155, Validation accuracy: 2.96% 
Epoch [112/200], Train Loss: 2.856440, Validation Loss: 3.144734, Validation accuracy: 2.95% 
Epoch [113/200], Train Loss: 2.856198, Validation Loss: 3.144632, Validation accuracy: 2.98% 
Model saved at epoch 114 with validation loss 3.144143 Learning rate: 1.00e-04 Weight decay: 1.00e-05 
Epoch [114/200], Train Loss: 2.856062, Validation Loss: 3.144143, Validation accuracy: 2.96% 
Epoch [115/200], Train Loss: 2.855804, Validation Loss: 3.144741, Validation accuracy: 2.96% 
Epoch [116/200], Train Loss: 2.855534, Validation Loss: 3.144215, Validation accuracy: 2.96% 
Epoch [117/200], Train Loss: 2.855416, Validation Loss: 3.145090, Validation accuracy: 2.93% 
Model saved at epoch 118 with validation loss 3.144

In [ ]:
# define function which update

In [38]:
write_charts_to_TB('Test data sample',writer, vlabels[0].cpu(), voutputs[0].cpu(), 100, index_to_candle)

In [80]:
#interrim model parameters and optimizer state saving

torch.save({
            'epoch': epoch,
            'batch_step': batch_idx,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'vloss': vepoch_loss,
            }, './models/nq-llm_0_interrim.pth')

In [27]:
# function which perform candle prediction in the following way:
# 1. take last 30 tockenized candles from test dataset
# 2. predict next tocken
# 3. add predicted tocken to the end of tockenized candles and remove first tocken
# 4. repeat 2-3 steps until new candle tocken (0) is predicted.
# 5. return predicted tockenized candle along with preceding 30 tockenized candles



In [ ]:
with torch.no_grad():
    for idx, (vdata, vlabels) in enumerate(test_loader):
        vdata = vdata.to(device)
        vlabels = vlabels.to(device)
        voutputs = model(vdata)    
        break 
F.softmax(voutputs,dim=1).argmax(dim=1), vlabels

In [60]:




#test the function
write_charts_to_TB('Test data sample',writer, vlabels.cpu(), voutputs.cpu(), 200, index_to_body)